# Clone git

In [1]:
!git clone https://github.com/ttharden/Keyframe-Extraction-for-video-summarization

Cloning into 'Keyframe-Extraction-for-video-summarization'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 197 (delta 30), reused 20 (delta 19), pack-reused 159 (from 1)
Receiving objects: 100% (197/197), 2.55 MiB | 32.70 MiB/s, done.
Resolving deltas: 100% (47/47), done.


# Import Thư viện

In [2]:
import sys
import os
import pickle
import numpy as np

# Thêm đường dẫn gốc

In [3]:
sys.path.append("/kaggle/working/Keyframe-Extraction-for-video-summarization/src/extraction")

# Đọc embeding

In [4]:
# embedding_path = '/kaggle/input/aic-sample-test/embeddings/L21_V006_features.pkl'
# shots_file = '/kaggle/input/aic-sample-test/shots_boundary/L21_V006.txt' 

In [5]:
# with open(embedding_path, "rb") as f:
#     embeddings = pickle.load(f)

# print(type(embeddings))   # numpy.ndarray
# print(embeddings.shape)   # số hàng, số chiều embedding
# print(embeddings[:2])     # in 2 vector đầu

# Đọc shot file

In [6]:
# def read_ranges(file_path):
#     """
#     Đọc file txt có dạng:
#         start end
#     Mỗi dòng chứa 2 số nguyên.
    
#     Args:
#         file_path (str): đường dẫn đến file .txt
    
#     Returns:
#         list[tuple[int, int]]: danh sách các cặp (start, end)
#     """
#     ranges = []
#     with open(file_path, "r", encoding="utf-8") as f:
#         for line in f:
#             line = line.strip()
#             if not line:  # bỏ dòng trống
#                 continue
#             parts = line.split()
#             if len(parts) == 2:
#                 start, end = map(int, parts)
#                 ranges.append((start, end))
#     return ranges


# # Ví dụ dùng:
# ranges = read_ranges(shots_file)
# print(ranges)
# # [(0, 52), (54, 342), (343, 410), ...]

# Gán nhãn cho embeddings theo shot

In [7]:
# len(embeddings)

In [8]:
# # Tổng số điểm embedding (ví dụ)
# n_samples = len(embeddings)  # bạn thay bằng len(embeddings)

# # Tạo mảng label mặc định = -1
# labels = np.full(n_samples, -1)

# # Gán nhãn cho từng khoảng
# for idx, (start, end) in enumerate(ranges):
#     labels[start:end+1] = idx   # end inclusive

# # Giờ labels[i] sẽ là nhãn của embedding i
# print(labels[:100])  # in thử 100 phần tử đầu

# Giảm chiều Umap Plot embedding

In [9]:
# import umap
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.preprocessing import StandardScaler
# import time

# start = time.time()
# # 2. Chuẩn hóa dữ liệu trước khi đưa vào UMAP
# X_scaled = StandardScaler().fit_transform(embeddings)

# # 3. Khởi tạo UMAP giảm chiều về 2D
# umap_model = umap.UMAP(
#     n_neighbors=50,     # số láng giềng gần, ảnh hưởng đến cấu trúc local
#     n_components=2,     # số chiều muốn giảm về (2 hoặc 3)
#     metric='euclidean', # khoảng cách sử dụng
# )

# X_umap = umap_model.fit_transform(X_scaled)
# end = time.time()
# print(f'Reduce embedding to plot time: {end - start}')

In [10]:
# X_umap.shape

In [11]:
# y = labels
# print(y)

In [12]:
# %matplotlib inline

# # 4. Vẽ scatter plot
# plt.figure(figsize=(8,6))
# sns.scatterplot(
#     x=X_umap[:,0], y=X_umap[:,1],
#     hue=y, palette="Spectral",
#     legend="full", s=30, alpha=0.8
# )
# plt.title("UMAP Projection of Digits Dataset")
# plt.savefig("umap_plot.png", dpi=300, bbox_inches="tight")
# print("Saved to umap_plot.png")

In [13]:
%%writefile /kaggle/working/Keyframe-Extraction-for-video-summarization/src/extraction/Keyframe_extraction.py
import pickle
import cv2
import numpy as np

from Kmeans_improvment import kmeans_silhouette
from Redundancy import redundancy
import sys
import os
import json

sys.path.append('/kaggle/working/Keyframe-Extraction-for-video-summarization/src/scripts')
from save_keyframe import save_frames

def scen_keyframe_extraction(scenes_path, features_path, video_path, save_path, folder_path):
    # Get lens segmentation data
    number_list = []
    with open(scenes_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            numbers = line.strip().split(' ')
            number_list.extend([int(number) for number in numbers])

    # Read inference data from local
    with open(features_path, 'rb') as file:
        features = pickle.load(file)

    features = np.asarray(features)
    # print(len(features))

    # Clustering at each shot to obtain keyframe sequence numbers
    keyframe_index = []
    for i in range(0, len(number_list) - 1, 2):
        start = number_list[i]
        end = number_list[i + 1]
        print(start, end)
        sub_features = features[start:end]
        best_labels, best_centers, k, index = kmeans_silhouette(sub_features)
        final_index = [x + start for x in index]
        final_index.sort()
        print("clustering：" + str(keyframe_index))
        print(start, end)
        final_index = redundancy(video_path, final_index, 0.94)
        print("final_index：", final_index)
        keyframe_index += final_index
    keyframe_index.sort()
    print("final_index：" + str(keyframe_index))

    # save keyframe
    save_frames(keyframe_index, video_path, save_path, folder_path)
    return keyframe_index

# ====== CHỈNH ĐƯỜNG DẪN ======
video_path = "/kaggle/input/aic-sample-test/videos/L21_V006.mp4"
scenes_path = "/kaggle/input/aic-sample-test/shots_boundary/L21_V006.txt"
features_path = "/kaggle/input/aic-sample-test/embeddings/L21_V006_features.pkl"

# Nơi lưu keyframes
save_path = "/kaggle/working"
folder_path = "video_keyframes"

os.makedirs(save_path, exist_ok=True)

keyframe_index = scen_keyframe_extraction(
    scenes_path=scenes_path,
    features_path=features_path,
    video_path=video_path,
    save_path=save_path,
    folder_path=folder_path
)

keyframes_index_file = '/kaggle/working/L21_V006_keyframes_index.json'

# Lưu mảng vào file JSON
with open(keyframes_index_file, 'w') as json_file:
    json.dump(keyframe_index, json_file)

print(f'Frameindexs đã được lưu vào file {file_path}')

print(f"✅ Keyframes đã lưu trong {os.path.join(save_path, folder_path)}")

Overwriting /kaggle/working/Keyframe-Extraction-for-video-summarization/src/extraction/Keyframe_extraction.py


In [14]:
%%writefile /kaggle/working/Keyframe-Extraction-for-video-summarization/src/extraction/init_center.py
import numpy as np
from tqdm import tqdm

def kmeans_init(data):
    print("🔹 In the process of initialising the center")
    n = len(data)
    sqrt_n = int(np.sqrt(n))           # số tâm cần chọn
    centers = []
    label = []

    # pick init_center
    while len(centers) < sqrt_n:
        sse_min = float('inf')
        join_center = data[0]

        # tqdm để quan sát tiến trình duyệt qua n điểm
        for i in tqdm(range(n), desc=f"Selecting center {len(centers)+1}/{sqrt_n}"):
            center = centers.copy()
            
            if len(centers) == 0 or not np.any(np.all(data[i] == centers, axis=1)):
                center.append(data[i])
                center = np.array(center)
                sse = 0.0

                # Cluster operation
                cluster_labels = np.zeros(len(data)).astype(int)
                for k in range(len(data)):
                    distances = [np.sqrt(np.sum((data[k] - cen) ** 2)) for cen in center]
                    nearest_cluster = np.argmin(distances)
                    cluster_labels[k] = nearest_cluster

                # Based on the results of the cluster operation, calculate sse
                for j in range(len(center)):
                    cluster_points = [data[l] for l in range(len(cluster_labels)) if cluster_labels[l] == j]
                    singe_sse = sum(np.linalg.norm(point - center[j]) for point in cluster_points)
                    sse += singe_sse

                if sse < sse_min:
                    sse_min = sse
                    join_center = data[i]
                    label = cluster_labels.copy()

        centers.append(join_center)

    return np.array(label), np.array(centers)

Overwriting /kaggle/working/Keyframe-Extraction-for-video-summarization/src/extraction/init_center.py


In [15]:
%%writefile /kaggle/working/Keyframe-Extraction-for-video-summarization/src/scripts/save_keyframe.py
import os
import cv2
from tqdm import tqdm  # Thêm tqdm

def save_frames(keyframe_indexes, video_path, save_path, folder_name):
    # Open video
    cap = cv2.VideoCapture(video_path)

    # Creating a folder path for saving images
    folder_path = os.path.join(save_path, folder_name)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Setting the current frame number
    current_index = 0

    # Cyclic reading of the video frames with tqdm
    for _ in tqdm(range(total_frames), desc="Processing frames", ncols=100):  # Add tqdm here
        ret, frame = cap.read()

        if not ret:
            break

        # Check if the current frame is a keyframe, if yes, save the frame image
        if current_index in keyframe_indexes:
            file_name = f"keyframes_{current_index:010d}.jpg"  # Fix variable name to 'current_index'
            file_path = os.path.join(folder_path, file_name)
            cv2.imwrite(file_path, frame)
        
        current_index += 1

    # release
    cap.release()

Overwriting /kaggle/working/Keyframe-Extraction-for-video-summarization/src/scripts/save_keyframe.py


In [16]:
!python /kaggle/working/Keyframe-Extraction-for-video-summarization/src/extraction/Keyframe_extraction.py

1 73
🔹 In the process of initialising the center
Selecting center 8/8: 100%|████████████████████| 72/72 [00:00<00:00, 184.37it/s]
clustering：[]
1 73
[6, 66]
final_index： [6, 66]
74 560
🔹 In the process of initialising the center
Selecting center 22/22: 100%|█████████████████| 486/486 [00:45<00:00, 10.61it/s]
clustering：[6, 66]
74 560
[423, 515, 532, 546]
final_index： [423, 515, 532, 546]
561 626
🔹 In the process of initialising the center
Selecting center 8/8: 100%|████████████████████| 65/65 [00:00<00:00, 207.22it/s]
clustering：[6, 66, 423, 515, 532, 546]
561 626
567 0.99952036 564
569 0.9992997 564
572 0.997973 564
574 0.9973421 564
579 0.9888003 564
611 0.9830418 564
[564]
final_index： [564]
627 666
🔹 In the process of initialising the center
Selecting center 6/6: 100%|████████████████████| 39/39 [00:00<00:00, 432.31it/s]
clustering：[6, 66, 423, 515, 532, 546, 564]
627 666
657 0.9975712 632
[632]
final_index： [632]
667 713
🔹 In the process of initialising the center
Selecting center